In [148]:
import pandas as pd
import numpy as np

In [149]:
dataOriginal = pd.read_csv("./inscripcion.tsv",sep="\t")
data = pd.read_csv("./inscripcion.tsv",sep="\t")

In [150]:
data.keys()

Index(['Marca temporal', 'Dirección de correo electrónico',
       'Nombre y apellidos', 'Posición principal', 'Otras posiciones',
       'Género', 'Equipo', 'Títulos de arbitraje', 'Años jugando',
       'Color favorito [Verde]', 'Color favorito [Rojo]',
       'Color favorito [Blanco]', 'Color favorito [Negro]'],
      dtype='object')

In [151]:
data["Género"].unique()

cont = 1
for gen in data["Género"].unique():
    data['Género'].mask(data['Género'] == gen, str(cont), inplace=True)
    cont+=1

In [152]:
data["Dirección de correo electrónico"] = ["@gmail.com"]*len(data["Dirección de correo electrónico"])

In [153]:
data.to_csv("./inscripcionClean.tsv")

In [154]:
import itertools

def makeTeams(data, amount):
    suffledData = data.sample(frac = 1)
    return np.array_split(suffledData,amount)

def satisfiesGenderRule(team):
    """
    Checks if the sum of genders can fulfill the
    gender rule.
    """
    return sumOfGendersDontBreakGender(team)>=7

def sumOfGendersDontBreakGender(team):
    """
    If the team allows sums all people by gender.
    Being 4 the maximum amount it can sum for each.
    
    This is used to evaluate dispersation
    """
    genders = team["Género"].value_counts()
    total = 0
    if genders:
        for gen in genders:
            total += min(gen,4)
    return total

def validCombByGender(team, showGenderId):
    """
    Return all combinations that fullfill the gender rule
    """
    cName = {"name":"Nombre y apellidos", "gender": "Género"}
    
#     name = team[colNames["name"]].values
#     gen = team[colNames["gender"]].values
    if showGenderId:
        team = team.assign(
            namegen = lambda row: row[cName["name"]] + " is " + row[cName["gender"]]
        )
        combsName = pd.DataFrame(list(itertools.combinations(team['namegen'].values,7)))
    else:
        combsName = pd.DataFrame(list(itertools.combinations(team[cName["name"]].values,7)))
        
    combsGen = pd.DataFrame(list(itertools.combinations(team[cName["gender"]].values,7)))
    combsGen.insert(0,'CountMax', combsGen.T.apply(pd.Series.duplicated).sum())
    
    combsValidas=combsName[combsGen['CountMax']<=4]
    return combsValidas.reset_index()

In [155]:
team=makeTeams(data, 5)[0]
validCombByGender(team,True)

,index,0,1,2,3,4,5,6
0,0,Rafael Pérez is 2,Eugenia Sáez Waldo is 4,Aitor Uriarte Rufrancos is 3,Ane Zabala is 4,Carlos Denis barros is 2,Mikel Salazar is 2,Andrew Davalos Morales is 2
1,1,Rafael Pérez is 2,Eugenia Sáez Waldo is 4,Aitor Uriarte Rufrancos is 3,Ane Zabala is 4,Carlos Denis barros is 2,Mikel Salazar is 2,Ane Aranguren Ubierna is 5
2,2,Rafael Pérez is 2,Eugenia Sáez Waldo is 4,Aitor Uriarte Rufrancos is 3,Ane Zabala is 4,Carlos Denis barros is 2,Mikel Salazar is 2,Víctor Cuesta Helguera is 2
3,3,Rafael Pérez is 2,Eugenia Sáez Waldo is 4,Aitor Uriarte Rufrancos is 3,Ane Zabala is 4,Carlos Denis barros is 2,Mikel Salazar is 2,Guillermo Ruiz Villegas is 6
4,4,Rafael Pérez is 2,Eugenia Sáez Waldo is 4,Aitor Uriarte Rufrancos is 3,Ane Zabala is 4,Carlos Denis barros is 2,Mikel Salazar is 2,Ángel Siendones Sillero is 2
...,...,...,...,...,...,...,...,...
748,787,Carlos Denis barros is 2,Mikel Salazar is 2,Andrew Davalos Morales is 2,Ane Aranguren Ubierna is 5,Guillermo Ruiz Villegas is 6,Ángel Siendones Sillero is 2,Ane Miren Blanco is 4
749,788,Carlos Denis barros is 2,Mikel Salazar is 2,Andrew Davalos Morales is 2,Víctor Cuesta Helguera is 2,Guillermo Ruiz Villegas is 6,Ángel Siendones Sillero is 2,Ane Miren Blanco is 4
750,789,Carlos Denis barros is 2,Mikel Salazar is 2,Ane Aranguren Ubierna is 5,Víctor Cuesta Helguera is 2,Guillermo Ruiz Villegas is 6,Ángel Siendones Sillero is 2,Ane Miren Blanco is 4
751,790,Carlos Denis barros is 2,Andrew Davalos Morales is 2,Ane Aranguren Ubierna is 5,Víctor Cuesta Helguera is 2,Guillermo Ruiz Villegas is 6,Ángel Siendones Sillero is 2,Ane Miren Blanco is 4


In [ ]:
def method1(maxIter, teamAmount):
    """
    Returns set of teams that has the best average 
    """
    bestTeams = makeTeam(data, teamAmount)
    bestCombSTD = [0]*teamAmount
    for x in range(maxIter):
        teams = makeTeams(data, teamAmount)
        combinationsLenghts = [0]*teamAmount
        for i in range(len(teams)):
            combinationsLenghts[i] = len(validCombByGender(teams[i], True))
        if np.average(combinationsLenghts)>np.average(bestCombSTD):
            bestCombSTD = combinationsLenghts
            bestTeams = teams
    return bestTeams,bestCombSTD

In [ ]:
maxIter = 100
teamAmount = 5

bestTeams,_ = method1(maxIter,teamAmount)
for x in bestTeams:
    x = x.assign(
            namegen = lambda row: row["Nombre y apellidos"] + " is " + row["Género"]
    )
    print(x["namegen"])